In [2]:
from sklearn.model_selection import ShuffleSplit
from skimage.feature import hog
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import itertools
import matplotlib.image as mpimg
import pandas as pd
import numpy as np
import copy
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import glob

import data
import model
import postprocess
%matplotlib inline

In [3]:
# Read in all the car and non-car classes
vehicle_data = data.retrieve_data("vehicles")
non_vehicle_data = data.retrieve_data("non-vehicles")

df = pd.concat([vehicle_data, non_vehicle_data])
df

,category,subcategory,image
0,vehicles,KITTI_extracted,training_images/vehicles/KITTI_extracted/1.png
1,vehicles,KITTI_extracted,training_images/vehicles/KITTI_extracted/10.png
2,vehicles,KITTI_extracted,training_images/vehicles/KITTI_extracted/1000.png
3,vehicles,KITTI_extracted,training_images/vehicles/KITTI_extracted/1001.png
4,vehicles,KITTI_extracted,training_images/vehicles/KITTI_extracted/1002.png
5,vehicles,KITTI_extracted,training_images/vehicles/KITTI_extracted/1003.png
6,vehicles,KITTI_extracted,training_images/vehicles/KITTI_extracted/1004.png
7,vehicles,KITTI_extracted,training_images/vehicles/KITTI_extracted/1005.png
8,vehicles,KITTI_extracted,training_images/vehicles/KITTI_extracted/1006.png
9,vehicles,KITTI_extracted,training_images/vehicles/KITTI_extracted/1007.png


In [4]:
# Shuffle and do train-test split

# TODO Later: Split the data manually so that time series issues do not impact the validation results
cv = ShuffleSplit(n_splits=1, test_size=0.2, random_state=0).split(df)

# Since n_splits is 1, generator should only produce one object
train_idx, test_idx = next(cv)

df['dataset'] = 'test'
df['dataset'].iloc[train_idx.tolist()] = 'train'


print("Length of data: ", len(df))

Length of data:  9866


In [5]:
# For optimizing the subset, we need to be able to rely on validation figures
# Hence, we only use KITTI_extracted data given that the other data sets have timelapse info
# (i.e. high correlation between some images) + non-vehicles images
subset_df = pd.concat([df[df['subcategory'] == 'KITTI_extracted'], df[df['category'] == 'non-vehicles']])

subset_df

,category,subcategory,image,dataset
0,vehicles,KITTI_extracted,training_images/vehicles/KITTI_extracted/1.png,train
1,vehicles,KITTI_extracted,training_images/vehicles/KITTI_extracted/10.png,train
2,vehicles,KITTI_extracted,training_images/vehicles/KITTI_extracted/1000.png,train
3,vehicles,KITTI_extracted,training_images/vehicles/KITTI_extracted/1001.png,train
4,vehicles,KITTI_extracted,training_images/vehicles/KITTI_extracted/1002.png,train
5,vehicles,KITTI_extracted,training_images/vehicles/KITTI_extracted/1003.png,train
6,vehicles,KITTI_extracted,training_images/vehicles/KITTI_extracted/1004.png,train
7,vehicles,KITTI_extracted,training_images/vehicles/KITTI_extracted/1005.png,train
8,vehicles,KITTI_extracted,training_images/vehicles/KITTI_extracted/1006.png,train
9,vehicles,KITTI_extracted,training_images/vehicles/KITTI_extracted/1007.png,test
